# Main compound analisis after compound dicovery step

Requirements: 
- Exported compounds database in excel format
- Exported Chemspider results database in excel format


In [1]:
import pandas as pd
from chemspi_local_db import ChemspiLocalDB
from chemspi_web_db import ChemspiWebDB

# Input
compounds_database_file = "plasma_melanoma_negative_in_nature.xlsx"
chemspider_results_db_file = "plasma_melanoma_negative_csdatabase.xlsx"

export_file = "plasma_melanoma_negative_in_nature_with_external_sources.xlsx"

web_api_key = "XcuJRmExG2G949vyni9P6GQJ54TkEZMo"


# init resources and databases
compounds_table = pd.read_excel(compounds_database_file)
chempider_local_db = ChemspiLocalDB(chemspider_results_db_file)
chempider_web_db = ChemspiWebDB(web_api_key)

# remove not needed columns
compounds_table.drop(columns=['Checked'], inplace=True)


print("Sucess reading databases")
#display(compounds_table)

Sucess reading databases


## Step 1: Removing duplicated entries
- Sort by Name
- Sort by area
- Sort by retention time
- Select from duplicated names: one with highest area then retention time

In [2]:
import compound_analisis_utils as utils
compounds_table = utils.remove_duplicated_entries(compounds_table)
print("Done")
#display(compounds_table)

Done


# Step 2: Find CSId from chemspi databases
- Try local first
- Try online otherwise

In [3]:
csid_values = []
for row_index, row_data in compounds_table.iterrows() :
    #print("#########################")
    compound_name = row_data.Name.lower()
    compound_mass = row_data['Molecular Weight']
    #print("Searching csid for compound: ", compound_name)
    db_entry = chempider_local_db.find_compound_by_name(compound_name)
    
    if (db_entry is not None) and (db_entry.Name.lower() == compound_name):
        #print("Found in Local DB:           ", db_entry.Name)
        #print("CSID:                        ", db_entry.CSID)
        csid_values.append(db_entry.CSID)
    else:
        #print("#########################")
        print("Compound(", compound_name, ") not found in local database, trying online...")
        compound_csids, warnings = chempider_web_db.find_compounds_by_name_mass(compound_name, compound_mass)
        if len(warnings) > 0:
            print("Warnings:", warnings)
            
        if len(compound_csids) > 1 :
            csids = ""
            for csid in compound_csids:
                csids += str(csid) + ";";
            print("Found csids: ", csids)
            csid_values.append(csids)
        elif len(compound_csids) == 1 :
            csid_values.append(compound_csids[0])
        else:
            #print("Not Found")
            csid_values.append('ND')

        
compounds_table.insert(0, 'CSID', csid_values)
display(compounds_table)
#compounds_table.to_excel(export_file)

Compound( n-({(2r,4s,5r)-5-[2-methyl-6-(2-thienyl)-4-pyrimidinyl]-1-azabicyclo[2.2.2]oct-2-yl}methyl)cyclobutanecarboxamide ) not found in local database, trying online...
Compound( perfluoro-1-octanesulfonic acid (pfos) ) not found in local database, trying online...
Warnings: ['Compound Not found in chemspider web search']
Compound( pentacosanoic acid ) not found in local database, trying online...
Compound( 11(z),14(z)-eicosadienoic acid ) not found in local database, trying online...
Compound( 16-hydroxyhexadecanoic acid ) not found in local database, trying online...
Compound( all-cis-4,7,10,13,16-docosapentaenoic acid ) not found in local database, trying online...
Found csids:  4445975;4945614;
Compound( n-phenylacetylglutamine ) not found in local database, trying online...
Warnings: ['Compound Not found in chemspider web search']
Compound( np-020292 ) not found in local database, trying online...
Warnings: ['Compound Not found in chemspider web search']
Compound( glycoursodeox

,CSID,Name,Formula,Annotation Source: Predicted Compositions,Annotation Source: mzCloud Search,Annotation Source: Metabolika Search,Annotation Source: ChemSpider Search,FISh Coverage,Molecular Weight,RT [min],...,Group CV [%]: 53,Group CV [%]: 54,Group CV [%]: 55,Group CV [%]: 56,Group CV [%]: 57,Group CV [%]: 58,Group CV [%]: 6,Group CV [%]: 7,Group CV [%]: 8,Group CV [%]: 9
0,483911,2-Chloro-2-methyloctahydro-1H-2-benzosilole,C9 H17 Cl Si,No results,No results,No results,Full match,NaN,188.07814,0.393,...,5.026042,22.865763,24.794925,54.637288,17.317780,7.044397,14.855329,8.188045,20.351583,52.788893
1,531430,"4-Chloro[1,2,3]thiadiazolo[4,5-e]benzofurazan",C6 H Cl N4 O S,No results,No results,No results,Full match,NaN,211.95617,12.338,...,27.637411,100.096191,21.397249,114.673742,17.396473,21.664932,56.025215,21.880902,13.134138,52.423262
2,117424,1-(5-Chloro-2-hydroxyphenyl)-3-[3-(trifluorome...,C14 H10 Cl F3 N2 O2,No results,No results,No results,Full match,NaN,330.03700,0.379,...,9.350012,16.729660,12.174925,16.103834,14.585308,14.161786,58.483670,5.038775,12.598761,5.475770
3,22798,Phosphorus trichloride,Cl3 P,No results,No results,No results,Full match,NaN,135.88008,15.632,...,52.861233,67.826814,2.189233,67.606130,33.757487,13.849001,42.076281,124.005414,10.052870,38.361888
4,231450,Alanylalanylalanine,C9 H17 N3 O4,No results,No results,No results,Full match,NaN,231.12113,4.164,...,14.984435,11.329104,6.220870,11.941633,23.092365,5.657448,4.500326,14.855071,4.330909,108.535654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,21381294,"1-{[(2R,4S,5S)-5-{[Benzyl(methyl)amino]methyl}...",C22 H30 N4 O S,Not the top hit,Full match,No results,No match,NaN,398.21242,9.701,...,9.559159,10.469263,1.713410,20.845625,19.245930,7.098825,16.012710,27.486884,13.519745,15.331970
292,75744,Acetaminophen glucuronide,C14 H17 N O8,Not the top hit,Full match,No results,Full match,NaN,327.09544,0.851,...,149.650670,153.682347,102.800852,127.041034,133.822816,112.782108,136.779227,44.933298,107.154228,140.101566
293,54833,Valsartan,C24 H29 N5 O3,Not the top hit,Full match,No results,Full match,NaN,435.22693,9.391,...,69.514090,135.781808,34.550074,114.375875,34.014644,24.057343,45.688166,6.111664,10.592466,23.143817
294,14319,Pseudouridine,C9 H12 N2 O6,Not the top hit,Full match,Partial match,Full match,NaN,244.06902,0.425,...,64.504320,17.453065,7.299171,12.280286,21.824344,44.368477,67.489689,51.413850,34.590037,44.083461


# Step 3: Find External references from chemspi web search

In [4]:
external_databases = ['Human Metabolome Database', 'KEGG', 'LipidMAPS']
external_db_info = dict()
for db in external_databases:
    external_db_info[db] = []
        
for row_index, row_data in compounds_table.iterrows() :
    # search for external references from known databases
    print("Finding external references for ", row_data.Name)
    external_databases_result = {}
    for db in external_databases:
        external_databases_result[db] = []
    
    for csid in utils.parse_generated_CSID(row_data.CSID):
        print("CSID: ", csid)
        external_refs, warnings = chempider_web_db.find_external_references(csid, external_databases)
        display(external_refs)
        
        if len(warnings) > 0 :
            display(warnings)
        
        for db, values in external_refs.items():
            external_databases_result[db].extend(values)
    
    for db, values in external_databases_result.items():
        str_result = ""
        for v in values :
            str_result += str(v) + ";"
        external_db_info[db].append(str_result)
        
insert_pos = 1
for db_name, value in external_db_info.items():
    compounds_table.insert(insert_pos, db_name, value)
    insert_pos += 1

display(compounds_table)
compounds_table.to_excel(export_file)
print("Done!")

Finding external references for  2-Chloro-2-methyloctahydro-1H-2-benzosilole
CSID:  483911


{'Human Metabolome Database': [], 'KEGG': [], 'LipidMAPS': []}

['No external references found']

Finding external references for  4-Chloro[1,2,3]thiadiazolo[4,5-e]benzofurazan
CSID:  531430


{'Human Metabolome Database': [], 'KEGG': [], 'LipidMAPS': []}

...

{'Human Metabolome Database': ['HMDB0000767', 'HMDB00767'],
 'KEGG': ['C02067'],
 'LipidMAPS': []}

Finding external references for  Deoxycholic acid
CSID:  59651436


{'Human Metabolome Database': ['HMDB0000478', 'HMDB00478'],
 'KEGG': [],
 'LipidMAPS': []}

,CSID,Human Metabolome Database,KEGG,LipidMAPS,Name,Formula,Annotation Source: Predicted Compositions,Annotation Source: mzCloud Search,Annotation Source: Metabolika Search,Annotation Source: ChemSpider Search,...,Group CV [%]: 53,Group CV [%]: 54,Group CV [%]: 55,Group CV [%]: 56,Group CV [%]: 57,Group CV [%]: 58,Group CV [%]: 6,Group CV [%]: 7,Group CV [%]: 8,Group CV [%]: 9
0,483911,,,,2-Chloro-2-methyloctahydro-1H-2-benzosilole,C9 H17 Cl Si,No results,No results,No results,Full match,...,5.026042,22.865763,24.794925,54.637288,17.317780,7.044397,14.855329,8.188045,20.351583,52.788893
1,531430,,,,"4-Chloro[1,2,3]thiadiazolo[4,5-e]benzofurazan",C6 H Cl N4 O S,No results,No results,No results,Full match,...,27.637411,100.096191,21.397249,114.673742,17.396473,21.664932,56.025215,21.880902,13.134138,52.423262
2,117424,,C13835;,,1-(5-Chloro-2-hydroxyphenyl)-3-[3-(trifluorome...,C14 H10 Cl F3 N2 O2,No results,No results,No results,Full match,...,9.350012,16.729660,12.174925,16.103834,14.585308,14.161786,58.483670,5.038775,12.598761,5.475770
3,22798,,,,Phosphorus trichloride,Cl3 P,No results,No results,No results,Full match,...,52.861233,67.826814,2.189233,67.606130,33.757487,13.849001,42.076281,124.005414,10.052870,38.361888
4,231450,,,,Alanylalanylalanine,C9 H17 N3 O4,No results,No results,No results,Full match,...,14.984435,11.329104,6.220870,11.941633,23.092365,5.657448,4.500326,14.855071,4.330909,108.535654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,21381294,,,,"1-{[(2R,4S,5S)-5-{[Benzyl(methyl)amino]methyl}...",C22 H30 N4 O S,Not the top hit,Full match,No results,No match,...,9.559159,10.469263,1.713410,20.845625,19.245930,7.098825,16.012710,27.486884,13.519745,15.331970
292,75744,HMDB0010316;HMDB10316;,,,Acetaminophen glucuronide,C14 H17 N O8,Not the top hit,Full match,No results,Full match,...,149.650670,153.682347,102.800852,127.041034,133.822816,112.782108,136.779227,44.933298,107.154228,140.101566
293,54833,HMDB0014323;HMDB14323;,D00400;,,Valsartan,C24 H29 N5 O3,Not the top hit,Full match,No results,Full match,...,69.514090,135.781808,34.550074,114.375875,34.014644,24.057343,45.688166,6.111664,10.592466,23.143817
294,14319,HMDB0000767;HMDB00767;,C02067;,,Pseudouridine,C9 H12 N2 O6,Not the top hit,Full match,Partial match,Full match,...,64.504320,17.453065,7.299171,12.280286,21.824344,44.368477,67.489689,51.413850,34.590037,44.083461


Done!
